In [2]:
import pandas as pd
import plotly.graph_objects as go
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np

In [3]:
dict_stocks = { 'ANA'  : 'Acciona S.A.', 
             'ACX'  : 'Acerinox S.A.', 
             'ACS'  : 'Actividades de Construcción y Servicios S.A.',
             'AENA' : 'AENA SME S.A.',
             'AMA'  : 'Amadeus IT Holding S.A.',
             'MTS'  : 'ArcelorMittal S.A.',
             'SABE' : 'Banco de Sabadell S.A.',
             'BKIA' : 'Bankia S.A.',
             'BKT'  : 'Bankinter S.A.',
             'BBVA' : 'Banco Bilbao Vizcaya Argentaria S.A.',
             'CABK' : 'CaixaBank S.A.',
             'CLNX' : 'Cellnex Telecom S.A.',
             'CIEA' : 'CIE Automotive S.A.',
             'COL'  : 'Inmobiliaria Colonial S.A.',
             'ENAG' : 'Enagás S.A.',
             'ENC'  : 'ENCE Energia y Celulosa SA',
             'ELE'  : 'Endesa S.A.',
             'FER'  : 'Ferrovial S.A.',
             'GRLS' : 'Grifols S.A.',
             'ICAG' : 'International Consolidated Airlines Group S.A.',
             'IBE'  : 'Iberdrola S.A.',
             'ITX'  : 'Industria de Diseño y Textil S.A.',
             'IDR'  : 'Indra Sistemas S.A.',
             'MAP'  : 'Mapfre S.A.',
             'MASM' : 'MásMóvil Ibercom S.A.',
             'TL5'  : 'Mediaset España Comunicación S.A.',
             'MEL'  : 'Meliá Hotels International S.A.',
             'MRL'  : 'Merlin Properties Socimi S.A.',
             'NTGY' : 'Naturgy Energy Group S.A.',
             'REE'  : 'Red Eléctrica de España S.A.U',
             'REP'  : 'Repsol S.A.',
             'SAN'  : 'Banco Santander S.A.',
             'SGREN': 'Siemens Gamesa Renewable Energy S.A.',
             'TEF'  : 'Telefónica S.A.',
             'VIS'  : 'Viscofan S.A.'
            }

# Fuente de datos Investing.es

In [27]:
def load_data_investing(stock):

    name_df = "df_" + stock.lower()
    path_csv = "data/" + stock
    files = [f for f in listdir(path_csv) if isfile(join(path_csv, f))]
    
    df_invest = pd.DataFrame()
    appended_data = []
    for file in files:
        if 'csv' in file:
            df = pd.read_csv(path_csv + "/" + file ,decimal=",",
                                              header=0,names=['Date', 'Close', 'Open','Max','Min','Vol','% var'])
            appended_data.append(df)
    
    df_invest = pd.concat(appended_data)
    # Convertimos el string en Date
    df_invest['Date'] = pd.to_datetime(df_invest['Date'],format="%d.%m.%Y")
    
    # Convertimos el string en float y cambiamos los - por Nan
    df_invest['Vol'] = df_invest['Vol'].str.replace(',','.')
    df_invest['Vol'] = df_invest['Vol'].replace('-', np.nan, regex=True)
   
    # % Var los convertimos en float
    df_invest['% var'] = df_invest['% var'].str.replace('%','').str.replace('.','').str.replace(',','.').astype(float)
    
    # Ordenamos el df por fecha ascendente
    df_invest = df_invest.sort_values(by=['Date'],ascending=True)
    
    return df_invest

In [11]:
df_stock = load_data_investing('SAN')
df_stock.dtypes

Date     datetime64[ns]
Close           float64
Open            float64
Max             float64
Min             float64
Vol              object
% var           float64
dtype: object

## Valor a Predecir

In [19]:
df_stock = load_data_investing('SAN')
df_stock.head(5)
#df_stock.dtypes

,Date,Close,Open,Max,Min,Vol,% var
0,1994-11-17,1.656,1.640,1.656,1.634,5.46M,0.36
1,1994-11-18,1.659,1.656,1.672,1.650,7.37M,0.18
2,1994-11-21,1.665,1.678,1.687,1.659,5.18M,0.36
3,1994-11-22,1.653,1.653,1.665,1.640,1.83M,-0.72
4,1994-11-23,1.646,1.628,1.650,1.624,5.61M,-0.42


In [148]:
def plot_stock(df_stock):
    # Create figure
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df_stock['Date'], y=df_stock['Close']))

    # Add range slider
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label="1m",
                         step="month",
                         stepmode="backward"),
                    dict(count=6,
                         label="6m",
                         step="month",
                         stepmode="backward"),
                    dict(count=1,
                         label="YTD",
                         step="year",
                         stepmode="todate"),
                    dict(count=1,
                         label="1y",
                         step="year",
                         stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        )
    )

    fig.update_layout(
        title="Santander Price History",
        xaxis_title="Date",
        yaxis_title="Price",
        width=1024,
        height=600,
        font=dict(
            family="Courier New, monospace",
            size=15,
            color="#7f7f7f"
        )
    )
    
    return fig

In [150]:
plot_stock(df_stock).show()

## Verificación de la data

### Banco Santander

Observando la gráfica vemos que el 9/6/2008 hay una caída de más de 35% del valor y el 1/1/2008 un subida del 58%

In [14]:
# Ordenados de forma ascendente para verificar las mayores caidas
df_stock.sort_values(by=['% var'],ascending=True).head(10)

,Date,Close,Open,Max,Min,Vol,% var
2127,2008-06-09,7.882,7.933,8.010,7.792,86.81M,-36.64
1013,2016-06-24,3.272,3.065,3.432,3.045,347.27M,-19.88
2230,2011-09-20,4.801,4.658,4.806,4.638,39.18M,-17.04
63,2020-03-12,2.279,2.619,2.625,2.269,224.41M,-16.82
738,1997-10-28,3.501,3.260,3.548,3.260,30.87M,-14.94
969,1998-10-01,3.610,4.054,4.054,3.610,42.34M,-14.80
1386,2015-01-09,5.566,5.832,5.859,5.460,475.61M,-14.09
1036,1999-01-13,4.586,5.017,5.043,4.432,40.36M,-12.03
66,2020-03-09,2.680,2.900,2.901,2.635,199.00M,-11.99
2216,2008-10-10,9.070,9.850,9.850,8.970,NaN,-11.94


In [15]:
# Ordenados de forma descendente para verificar las mayores subidas
df_stock.sort_values(by=['% var'],ascending=False).head(10)

,Date,Close,Open,Max,Min,Vol,% var
2128,2008-06-10,12.460,12.120,12.570,12.020,NaN,58.08
2014,2008-01-01,14.790,14.790,14.790,14.790,NaN,55.67
2582,2010-05-10,9.500,9.200,9.500,8.900,NaN,23.22
1039,1999-01-18,5.405,5.405,5.405,5.303,24.52M,14.95
2229,2008-10-29,7.660,7.370,7.660,7.170,NaN,14.33
739,1997-10-29,3.984,3.804,3.984,3.676,30.18M,13.80
2289,2009-01-28,6.520,5.750,6.570,5.710,NaN,13.39
2201,2008-09-19,11.240,11.250,11.590,10.480,NaN,12.96
2318,2009-03-10,4.500,4.010,4.500,3.950,NaN,12.50
2217,2008-10-13,10.190,9.800,10.190,9.420,NaN,12.35


In [16]:
# Verificamos que % de volumen no tenemos
(len(df_stock) - df_stock['Vol'].count())/len(df_stock)*100

11.040765904879555

---
#### Hay un par de outlier que podríamos eliminar pero me falta el 11% del campo volumen
---

### Telefónica

In [20]:
df_stock = load_data_investing('TEF')

In [21]:
df_stock.sort_values(by=['% var'],ascending=True).head(10)

,Date,Close,Open,Max,Min,Vol,% var
1013,2016-06-24,7.850,7.577,8.514,7.564,71.35M,-16.11
63,2020-03-12,3.800,4.165,4.237,3.783,67.38M,-14.01
66,2020-03-09,4.593,4.779,4.915,4.593,48.90M,-9.82
3063,1997-10-28,5.758,5.159,5.758,5.159,61.81M,-9.64
18,2020-05-19,3.900,4.362,4.378,3.881,49.43M,-9.43
2847,1998-09-10,7.949,8.651,8.666,7.770,34.39M,-9.41
226,2009-02-10,13.389,13.594,13.817,13.371,23.43M,-9.29
2598,2010-04-16,16.369,16.713,16.783,16.356,46.77M,-9.11
308,2008-10-10,12.822,12.850,13.296,12.227,114.57M,-9.10
300,2008-10-22,13.250,14.087,14.105,13.036,82.35M,-8.83


In [22]:
df_stock.sort_values(by=['% var'],ascending=False).head(10)

,Date,Close,Open,Max,Min,Vol,% var
60,2020-03-17,4.338,3.888,4.413,3.652,31.49M,17.82
1878,2002-07-25,8.490,8.624,8.651,8.151,104.24M,14.17
2524,2010-07-29,17.690,17.250,17.690,17.240,NaN,11.88
295,2008-10-29,13.110,12.627,13.110,12.469,77.04M,10.75
1892,2002-07-05,7.972,7.320,7.972,7.275,65.62M,10.68
133,2009-06-24,16.020,15.610,16.020,15.530,NaN,10.16
181,2009-04-16,15.340,15.130,15.340,15.050,NaN,9.62
307,2008-10-13,14.050,13.715,14.050,12.971,72.53M,9.58
2439,2000-05-02,22.078,21.302,22.202,20.683,36.99M,9.23
109,2009-07-28,17.430,17.210,17.450,17.200,NaN,8.79


In [23]:
(len(df_stock) - df_stock['Vol'].count())/len(df_stock)*100

6.318553993511509

---
#### Teléfonica los datos son buenos y solo nos falta un 6% del campo volumen
---

# Fuentes de datos Yahoo

#### Cogemos otra fuente de datos para completar la información que nos falta y comparar los valores de cierre, máximo y mínimo del día

In [207]:
def load_data_yahoo(stock):

    name_df = "df_" + stock.lower()
    path_csv = "data/" + stock + "/yahoo"
    
    files = [f for f in listdir(path_csv) if isfile(join(path_csv, f))]
    
    df_yahoo = pd.DataFrame()
    appended_data = []
    for file in files:
        if 'csv' in file:
            df = pd.read_csv(path_csv + "/" + file ,decimal=","
                             ,header=0,
                             names=['Date','Open','Max','Min','Close','Adj Close','Vol'])
            appended_data.append(df)

    df_yahoo = pd.concat(appended_data)
    
    # Convertimos el string en Date
    df_yahoo['Date'] = pd.to_datetime(df_yahoo['Date'],format="%Y-%m-%d")
    df_yahoo['Close'] = df_yahoo['Close'].astype(float)
    df_yahoo['Open'] = df_yahoo['Open'].astype(float)
    df_yahoo['Max'] = df_yahoo['Max'].astype(float)
    df_yahoo['Min'] = df_yahoo['Min'].astype(float)
    df_yahoo['Adj Close'] = df_yahoo['Adj Close'].astype(float)
    
    df_yahoo = df_yahoo.sort_values(by=['Date'],ascending=True)
    
    # En la data de yahoo no viene % Var, lo calculamos y se lo añadimos al dataframe
    for i in range(1, len(df_yahoo)):
        df_yahoo.loc[i, '% var'] = ((df_yahoo_san.loc[i, 'Close']*100) / df_yahoo_san.loc[i-1, 'Close'])-100
    
    return df_yahoo

In [208]:
df_yahoo_san = load_data_yahoo('SAN')

In [209]:
df_yahoo_san.dtypes

Date         datetime64[ns]
Open                float64
Max                 float64
Min                 float64
Close               float64
Adj Close           float64
Vol                 float64
% var               float64
dtype: object

In [210]:
df_yahoo_san.head(5)

,Date,Open,Max,Min,Close,Adj Close,Vol,% var
0,2000-01-03,10.38600,10.42210,10.16020,10.2776,3.003019,8430823.0,NaN
1,2000-01-04,10.15110,10.20530,9.94343,10.0337,2.931755,8443928.0,-2.373122
2,2000-01-05,9.84409,9.97052,9.70862,9.7899,2.860519,8944664.0,-2.429812
3,2000-01-06,9.78990,9.78990,9.78990,9.7899,2.860519,0.0,0.000000
4,2000-01-07,9.86215,10.25050,9.85312,10.2234,2.987184,9203046.0,4.428033


In [211]:
(len(df_yahoo_san) - df_yahoo_san['Vol'].count())/len(df_yahoo_san)*100

0.09505703422053231

In [212]:
plot_stock(df_yahoo_san).show()

In [213]:
df_yahoo_san.sort_values(by=['% var'],ascending=False).head(10)

,Date,Open,Max,Min,Close,Adj Close,Vol,% var
2671,2010-05-10,8.62102,9.20221,8.62102,9.20221,4.490401,347478252.0,23.216600
2287,2008-10-29,6.05095,6.91795,6.05095,6.91795,2.957469,147315412.0,14.328329
2347,2009-01-28,5.56976,6.36405,5.53101,6.31562,2.749151,137176391.0,13.391241
2259,2008-09-19,8.98612,10.46720,8.98612,10.15110,4.339662,285303695.0,12.964216
2376,2009-03-10,3.87461,4.35894,3.82618,4.35894,1.936305,126838346.0,12.500097
2275,2008-10-13,8.19136,9.20287,8.19136,9.20287,3.934288,140273135.0,12.348499
5199,2020-03-24,2.21000,2.36900,2.18200,2.36900,2.259552,90207011.0,12.009456
5249,2020-06-05,2.33000,2.53000,2.32800,2.52700,2.527000,171612196.0,11.346111
3241,2012-07-26,4.01291,4.43584,4.01291,4.43584,2.659426,99577882.0,10.674927
2315,2008-12-08,5.90879,6.54810,5.90879,6.53841,2.846131,85336912.0,10.655650


In [214]:
df_yahoo_san.sort_values(by=['% var'],ascending=True).head(10)

,Date,Open,Max,Min,Close,Adj Close,Vol,% var
4240,2016-06-24,3.11787,3.49162,3.09820,3.32835,2.699923,374174676.0,-19.886437
5191,2020-03-12,2.61900,2.62500,2.26900,2.27850,2.173233,224408705.0,-16.858238
3867,2015-01-09,6.06952,6.09805,5.68200,5.79314,4.302182,531339537.0,-14.089921
5188,2020-03-09,2.90000,2.90100,2.63450,2.68000,2.556184,198998279.0,-11.972409
2274,2008-10-10,9.30221,9.30221,8.10105,8.19136,3.501860,175950467.0,-11.941786
671,2002-07-30,6.07805,6.80055,6.05998,6.07805,1.935051,79145540.0,-10.742660
5193,2020-03-16,2.19000,2.19900,1.92840,2.01800,1.924768,197503993.0,-10.609081
2284,2008-10-24,7.18889,7.18889,5.96064,6.45736,2.760563,353234611.0,-10.175841
2302,2008-11-19,5.82161,5.82161,5.24042,5.24042,2.281123,135797826.0,-9.983321
2282,2008-10-22,8.38102,8.38102,7.45080,7.55014,3.227735,217584205.0,-9.913829
